In [27]:
from easyquant import MongoIo
from easyquant.indicator.base import *
import easyquotation
import math
import numpy as np
import pandas as pd
from easyquant import MongoIo
# from easyquant.qafetch import QATdx as tdx
from easyquant.indicator.base import *
import pandas as pd
from easyquant import EasyTime
from datetime import datetime, time
from ctypes import * 
# import datetime
import math
import numpy as np
import pandas as pd
# from ctypes import * 
# import time

In [3]:
m=MongoIo()

In [ ]:
datam=m.get_stock_day('000758',st_start='2020-02-01', st_end='2020-12-02')

In [ ]:
C=datam.close
L=datam.low
H=datam.high
O=datam.open
VOL=datam.vol
AMOUNT=datam.amount

In [ ]:
P1=6;
P2=4;
P3=13;
A=(3*C+L+O+H)/6;
SSRCJL=MA(VOL,P3);
SSRCJE=MA(AMOUNT,P3);
SSRCBJX=((SSRCJE / SSRCJL) / 100);
SSRGL=(((A - SSRCBJX) / SSRCBJX) * 100);


In [ ]:
datam.iloc[-1].name[1]

In [ ]:
datam.query('code=="600718"')

In [ ]:
datam.index.names

In [ ]:
datam.groupby(level=['date']).sum()

In [ ]:
data.index.names

In [ ]:
result=pd.DataFrame()
for x in datam.index.levels[1]:
    qs="code=='%s'" % x
    data=datam.query(qs)
    CLOSE=data.close
    C=data.close
    前炮 = CLOSE > REF(CLOSE, 1) * 1.099
    小阴小阳 = HHV(ABS(C - REF(C, 1)) / REF(C, 1) * 100, 0) < 9
    时间限制 = IFAND(COUNT(前炮, 30) == 1, BARSLAST(前炮) > 5, True, False)
    后炮 = IFAND(REF(IFAND(小阴小阳, 时间限制, 1, 0), 1) , 前炮, 1, 0)
#     x.loc[:,'start_time'] = pd.to_datetime(x.loc[:,'start_time'])
    data['bflg'] = 后炮
#     nhp=pd.DataFrame(后炮)
#     data[:,'bflg'] = pd.Dataframe({'ss':后炮}
#     data['add_column']  = 1 
    if len(result) == 0:
        result=data
#         pass
    else:
        result=result.append(data)
#         pass
    
    

In [ ]:
len(result)

In [ ]:
result.query('code=="600756"')

In [4]:
data=m.get_stock_day('300620',st_start='2020-02-01', st_end='2020-12-02')


In [5]:
CLOSE=data.close
HIGH=data.high
LOW=data.low

VAR1G=(((CLOSE / MA(CLOSE,40)) * 100) < 78)
VAR2G=(((CLOSE / MA(CLOSE,60)) * 100) < 74)
VAR3G=(HIGH > (LOW * 1.051))
VAR4G=IFAND(VAR3G , (COUNT(VAR3G,5) > 1), 1, 0)
V3=(CLOSE-LLV(LOW,8))/(HHV(HIGH,8)-LLV(LOW,8))*100

买入信号=SMA(V3,2,1)
XH10=IFAND(买入信号>REF(买入信号,1) , REF(买入信号,1)<REF(买入信号,2) ,1,0)

XH1=IFAND(XH10, 买入信号<23 ,1.5,0)


In [6]:
LC=REF(CLOSE,1)
RSI5=((SMA(MAX((CLOSE - LC),0),5,1) / SMA(ABS((CLOSE - LC)),5,1)) * 100)
TR1=SUM(MAX(MAX((HIGH - LOW),ABS((HIGH - REF(CLOSE,1)))),ABS((LOW - REF(CLOSE,1)))),10)
HD=(HIGH - REF(HIGH,1))
LD=(REF(LOW,1) - LOW)

DMP=SUM(IF(((HD > 0) AND (HD > LD)),HD,0),10)
DMM=SUM(IF(((LD > 0) AND (LD > HD)),LD,0),10)
PDI=((DMP * 100) / TR1)
MDI=((DMM * 100) / TR1)
ADX=MA(((ABS((MDI - PDI)) / (MDI + PDI)) * 100),5)
AV=(RSI5 + ADX)
WR10=((100 * (HHV(HIGH,10) - CLOSE)) / (HHV(HIGH,10) - LLV(LOW,10)))
NEWVOL=(RSI5 - WR10)
回家种田=(AV + NEWVOL)
XH2=IF(CROSS(回家种田,0),1,0),STICK,COLORRED;
A=COUNT(CROSS(MA(C,60),MA(C,5)),7)=1;{7日内5日线跌破60日均线}
B=MA(C,5)<REF(MA(C,5),3) AND MA(C,5)>REF(MA(C,5),1){5日均线拐头}
XH21=A AND B AND V>REF(V,1)*1.3


SyntaxError: invalid syntax (<ipython-input-6-ea591a725491>, line 7)

In [ ]:

DMP=SUM(IF(((HD > 0) AND (HD > LD)),HD,0),10)
DMM=SUM(IF(((LD > 0) AND (LD > HD)),LD,0),10)
PDI=((DMP * 100) / TR1)
MDI=((DMM * 100) / TR1)
ADX=MA(((ABS((MDI - PDI)) / (MDI + PDI)) * 100),5)
AV=(RSI5 + ADX)
WR10=((100 * (HHV(HIGH,10) - CLOSE)) / (HHV(HIGH,10) - LLV(LOW,10)))
NEWVOL=(RSI5 - WR10)
回家种田=(AV + NEWVOL)
XH2=IF(CROSS(回家种田,0),1,0),STICK,COLORRED;
A=COUNT(CROSS(MA(C,60),MA(C,5)),7)=1;{7日内5日线跌破60日均线}
B=MA(C,5)<REF(MA(C,5),3) AND MA(C,5)>REF(MA(C,5),1){5日均线拐头}
XH21=A AND B AND V>REF(V,1)*1.3


P1=6
P2=4
P3=13
AD=(3*C+L+O+H)/6

VAR1=LLV(LOW,P1)
VAR2=HHV(HIGH,P1)
QS=WMA((AD-VAR1)/(VAR2-VAR1)*100,P2)
慢线=WMA(0.667*REF(QS,1)+0.333*QS,2)

GW=REF(QS,1)>80 OR QS>80
DW=REF(QS,1)<30 OR QS<30
XH3=CROSS(QS,慢线) AND DW

ZXH:(XH1 AND XH2) OR (XH1 AND XH3) OR (XH2 AND XH3)
ZXH2:XH3 AND XH21



In [28]:
len(TJ1)

105

In [8]:
data=m.get_stock_day('000758',st_start='2020-02-01', st_end='2020-12-02')
CLOSE=data.close
C=data.close
前炮 = CLOSE > REF(CLOSE, 1) * 1.099
小阴小阳 = HHV(ABS(C - REF(C, 1)) / REF(C, 1) * 100, BARSLAST(前炮)) < 9
# 小阴小阳 = HHV(ABS(C - REF(C, 1)) / REF(C, 1) * 100, 0) < 9
TJ1=ABS(C-REF(C,1))/REF(C,1)*100
TJ2=BARSLAST(前炮)
XYXY=HHV(TJ1,TJ2);
# 小阴小阳:HHV(ABS(C-REF(C,1))/REF(C,1)*100,BARSLAST(前炮))<9;

时间限制 = IFAND(COUNT(前炮, 30) == 1, BARSLAST(前炮) > 5, True, False)
后炮 = IFAND(REF(IFAND(小阴小阳, 时间限制, 1, 0), 1) , 前炮, True, False)
后炮[-1]

False

In [9]:
TJ1.tail(30)

date        code  
2020-05-21  000758    1.960784
2020-05-22  000758    0.480769
2020-05-25  000758    5.502392
2020-05-26  000758    0.253165
2020-05-27  000758    1.767677
2020-05-28  000758    9.925558
2020-05-29  000758    6.320542
2020-06-01  000758    0.849257
2020-06-02  000758    2.355460
2020-06-03  000758    1.315789
2020-06-04  000758    0.888889
2020-06-05  000758    1.345291
2020-06-08  000758    2.045455
2020-06-09  000758    1.559020
2020-06-10  000758    2.036199
2020-06-11  000758    2.309469
2020-06-12  000758    2.483070
2020-06-15  000758    3.935185
2020-06-16  000758    3.614458
2020-06-17  000758    0.930233
2020-06-18  000758    1.643192
2020-06-19  000758    0.923788
2020-06-22  000758    0.457666
2020-06-23  000758    0.229885
2020-06-24  000758    3.225806
2020-06-29  000758    1.666667
2020-06-30  000758    1.210654
2020-07-01  000758    3.349282
2020-07-02  000758    3.935185
2020-07-03  000758    3.118040
Name: close, dtype: float64

In [1]:
TJ2.tail(30)

NameError: name 'TJ2' is not defined

In [ ]:
前炮 = CLOSE > REF(CLOSE, 1) * 1.099
BARSLAST(前炮).tail()

In [ ]:
前炮:=CLOSE>REF(CLOSE,1)*1.099;
小阴小阳:=HHV(ABS(C-REF(C,1))/REF(C,1)*100,20)<9;
时间限制:=COUNT(前炮,30)=1 AND BARSLAST(前炮)>5;
后炮:REF(小阴小阳 AND 时间限制,1) AND 前炮;


In [ ]:
data=m.get_stock_day('300620',st_start='2020-02-01', st_end='2020-12-02')
CLOSE = data.close
C = data.close
前炮 = CLOSE > REF(CLOSE, 1) * 1.099
c1=ABS(C-REF(C,1))/REF(C,1)*100
c2=BARSLAST(前炮)
小阴小阳 = HHV(c1,c2)<9
# HHV(ABS(C - REF(C, 1)) / REF(C, 1) * 100, BARSLAST(前炮)) < 9
时间限制 = IFAND(COUNT(前炮, 30) == 1, BARSLAST(前炮) > 5, True, False)
后炮 = IFAND(REF(IFAND(小阴小阳, 时间限制, 1, 0), 1) , 前炮, True, False)
后炮.tail()

In [ ]:
c2.tail(25)

In [ ]:
小阴小阳.tail(25)

In [ ]:
c1.tail(22)

In [ ]:
c2.tail()

In [ ]:
HHV(ABS(C - REF(C, 1)) / REF(C, 1) * 100, 0).tail()

In [ ]:
后炮[-1]

In [ ]:
lib  = cdll.LoadLibrary("/usr/share/talib/%s" % ("talib_ext.so"))

def HHV2(Series, NS):
    ncount = len(NS)
    tf_p=c_float * ncount
    np_OUT =tf_p(0)
    na_Series=np.asarray(Series).astype(np.float32)
    na_NS=np.asarray(NS).astype(np.int32)
    
    np_S=cast(na_Series.ctypes.data, POINTER(c_float))
    np_N=cast(na_NS.ctypes.data, POINTER(c_int))
    
    lib.llv(ncount, np_OUT, np_S, np_N)
    
    return pd.Series(np.asarray(np_OUT), index=Series.index)

TJ1 = ABS(C - REF(C, 1)) / REF(C, 1) * 100
TJ2 = BARSLAST(前炮)
小阴小阳 = HHV2(TJ1, TJ2)
小阴小阳.tail(30)

In [ ]:
TJ1.tail(30)

In [ ]:
data[:,'flg']='a'

In [ ]:
TJ2.tail(30)

In [ ]:
result=pd.DataFrame()
result=后炮

In [ ]:
result.append(后炮)

In [ ]:
result.index.levels[0]

In [ ]:
position_col = data.columns.get_loc('low')

In [ ]:
position_col

In [ ]:
data.columns.data.columns

In [ ]:
N = pd.Series(np.full(len(Series),1), index=Series.index)

In [ ]:
b=Series.reset_index()

In [ ]:
b.diff(1, axis=1)

In [ ]:
REF(IFAND(小阴小阳, 时间限制, 1, 0), 1)

In [ ]:
B.tail()

In [ ]:
小阴小阳.tail()

In [ ]:
REF(小阴小阳, 1)

In [ ]:
N = pd.Series(np.full(len(Series),1))

In [ ]:
N

In [ ]:
Series=小阴小阳

In [ ]:
Series.reset_index().diff(N)

In [ ]:
b

In [ ]:
FLG=IFAND(REF(IFAND(小阴小阳, 时间限制, 1, 0), 1) , 前炮, True, False)

In [ ]:
pd.DataFrame({'FLG':FLG}).iloc[-1]['FLG']

In [ ]:
s = pd.Series(np.random.randint(-10,10,7), 
              index=[['a','a','b','b','c','c','c'],[0,1,0,1,0,1,2]])


In [ ]:
s.index

In [ ]:
type(df.A)

In [ ]:
s.index[:0]

In [ ]:
for _, c in s.index:
    print(c)

In [ ]:
codelist = ["600109", "600551", "600697", "601066", "000732", "000905", "002827","600338","002049","300620"]
st_start="2020-03-01"
data_day = m.get_stock_day(codelist, st_start=st_start)

In [ ]:
dataR = pd.DataFrame()
for code in datam.index.levels[1]:
    data1=datam.query("code=='%s'" % code)
    data=data1.copy()
    data['ret'] = data.close.pct_change(1)
    data['nav'] = (1+data.close.pct_change(1).fillna(0)).cumprod()
    if len(dataR) == 0:
        dataR = data
    else:
        dataR = dataR.append(data)

In [ ]:
dataR.sort_index().tail()

In [ ]:
len(datam.index.levels[1])

In [ ]:
dataR.groupby(level=['date']).sum().tail() - 1

In [ ]:
def pre_rsrs_data_v1_func(datam, N=18, M=252):
    start_t = datetime.datetime.now()
    print(start_t)
    dataR = pd.DataFrame()
    for code in datam.index.levels[1]:
        data=datam.query("code=='%s'" % code)
        CLOSE=data.close
        C=data.close
        前炮 = CLOSE > REF(CLOSE, 1) * 1.099
        小阴小阳 = HHV(ABS(C - REF(C, 1)) / REF(C, 1) * 100, BARSLAST(前炮)) < 9
        时间限制 = IFAND(COUNT(前炮, 30) == 1, BARSLAST(前炮) > 5, True, False)
        后炮 = IFAND(REF(IFAND(小阴小阳, 时间限制, 1, 0), 1) , 前炮, 1, 0)
        # 斜率
        data = data.copy()
        data['bflg'] = 后炮
        data['ret'] = data.close.pct_change(1)

        if len(dataR) == 0:
            dataR = data
        else:
            dataR = dataR.append(data)
    end_t = datetime.datetime.now()
    print(end_t, 'spent:{}'.format((end_t - start_t)))
    return dataR.sort_index()


In [ ]:
indices_rsrsT = pre_rsrs_data_v1_func(data_day)

In [ ]:
datam=indices_rsrsT

In [ ]:
dataR = pd.DataFrame()
for code in datam.index.levels[1]:
    # data = price.copy()
    price = datam.query("code=='%s'" % code)
    data = price.copy()
    data['flag'] = 0 # 买卖标记
    data['position'] = 0 # 持仓标记
    data['hprice'] = 0  # 持仓价格
    bflag = data.columns.get_loc('bflg')
    # beta = data.columns.get_loc('beta')
    flag = data.columns.get_loc('flag')
    position_col = data.columns.get_loc('position')
    close_col = data.columns.get_loc('close')
    open_col = data.columns.get_loc('open')
    hprice_col = data.columns.get_loc('hprice')
    position = 0 # 是否持仓，持仓：1，不持仓：0
    for i in range(1,data.shape[0] - 1):
        # 开仓
        if data.iat[i, bflag] > 0 and position == 0:
            data.iat[i + 1, flag] = 1
            data.iat[i + 1, position_col] = 1
            data.iat[i + 1, hprice_col] = data.iat[i+1, open_col]
            position = 1
#             print("buy  : date=%s code=%s price=%.2f" % (data.iloc[i+1].name[0], data.iloc[i+1].name[1], data.iloc[i+1].close))
        # 平仓
        # elif data.iat[i, bflag] == S2 and position == 1:
        elif data.iat[i, position_col] > 0 and position == 1:
            cprice = data.iat[i, close_col]
            # oprice = data.iat[i, open_col]
            hprice = data.iat[i, hprice_col]
            if cprice < hprice * 0.95 or cprice > hprice * 1.2:
                data.iat[i, flag] = -1
                data.iat[i + 1, position_col] = 0
                data.iat[i + 1, hprice_col] = 0
                position = 0
#                 print("sell : code=%s date=%s  price=%.2f" % (data.iloc[i].name[0], data.iloc[i].name[1], data.iloc[i].close))
            else:
                data.iat[i + 1, position_col] = data.iat[i, position_col]
                data.iat[i + 1, hprice_col] = data.iat[i, hprice_col]
        # 保持
        else:
            data.iat[i + 1, position_col] = data.iat[i, position_col]
            data.iat[i + 1, hprice_col] = data.iat[i, hprice_col]

    data['nav'] = (1+data.close.pct_change(1).fillna(0) * data.position).cumprod()
    # if code == '000732':
    #     print(data.tail(22))
    if len(dataR) == 0:
        dataR = data
    else:
        dataR = dataR.append(data)


In [ ]:
a=dataR.sort_index()

In [ ]:
type(a.index.levels[0])

In [ ]:
pd.to_datetime('2020-12-12')

In [ ]:
a.loc[pd.to_datetime('2020-04-27'):]

In [ ]:
result = dataR['nav'].groupby(level=['date']).sum()
result3 = dataR['nav'].groupby(level=['date']).count()
result2 = dataR.groupby(level=['date'])

In [ ]:
min(result)

In [ ]:
r1=result - result3 + 1

In [ ]:
min(r1)

In [ ]:
num = dataR.flag.abs().sum()

In [ ]:
num

In [ ]:
dataR2 = pd.DataFrame({'nav':r1,'flg':0})

In [ ]:
dataR2 = pd.DataFrame(r1)

In [ ]:
# dataR2['num'] = 0
dataR2['flg'].iloc[-1] = num

In [ ]:
dataR2.nav[39:51]

In [68]:
datam=m.get_index_day('512880',st_start='2020-02-01', st_end='2020-12-02')

In [23]:
df=pd.concat([MA(datam.close, x) for x in (5,10,20,30,60,90,120,250,13, 34, 55,) ], axis = 1)
df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250', u'm13', u'm34', u'm55']

In [12]:
MA(datam.close, 5)[-1]

1.2800000000000002

In [13]:
MA(datam.close, 10)[-1]

1.3075000000000006

In [14]:
MA(datam.close, 20)[-1]

1.1894999999999996

In [25]:
df.m5.iloc[-1]

1.2800000000000002

In [28]:
eq=easyquotation.use('sina')

In [31]:
datajs=eq.stocks('512880')

In [34]:
data=datajs['512880']

In [40]:
last_time = pd.to_datetime(data['datetime'][0:10])

In [36]:
last_time

Timestamp('2020-07-21 00:00:00')

In [37]:
now_price = data['now']

In [38]:
now_price

1.256

In [70]:
last_time = pd.to_datetime(data['datetime'][0:10])

In [69]:
df_day=datam

In [72]:
df_day.tail()

,,amount,close,high,low,open,vol
date,code,,,,,,
2020-07-15,512880,3.565837e+09,1.315,1.384,1.310,1.371,26544904.0
2020-07-16,512880,3.858486e+09,1.246,1.346,1.241,1.313,29684070.0
2020-07-17,512880,4.037402e+09,1.207,1.268,1.188,1.258,33008468.0
2020-07-20,512880,3.357121e+09,1.276,1.293,1.224,1.231,26747316.0
2020-07-21,512880,NaN,NaN,NaN,NaN,1.275,NaN


In [44]:
df_day.loc[last_time]=[0 for x in range(len(df_day.columns))]

In [71]:
df_day.loc[(last_time,'512880'),'open'] = data['open']

In [61]:
df_day.loc[('2020-07-20','512880')]

amount    3.357121e+09
close     1.276000e+00
high      1.293000e+00
low       1.224000e+00
open      1.231000e+00
vol       2.674732e+07
Name: (2020-07-20 00:00:00, 512880), dtype: float64

In [63]:
df_day.loc[(last_time,'512880')]=[0 for x in range(len(df_day.columns))]

ValueError: cannot set using a multi-index selection indexer with a different length than the value